In [188]:
import pandas as pd
import numpy as np
import requests
import json
import pickle
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator, TransformerMixin

from flask import Blueprint, request, jsonify, render_template, Flask
from flask_cors import CORS, cross_origin


In [189]:
class My_encoder(BaseEstimator, TransformerMixin):
   
    def __init__(self,drop = 'first',sparse=False):
        self.encoder = OneHotEncoder(drop = drop,sparse = sparse)
        self.features_to_encode = []
        self.columns = []
    
    def fit(self,X_train,features_to_encode):
        
        data = X_train.copy()
        self.features_to_encode = features_to_encode
        data_to_encode = data[self.features_to_encode]
        self.columns = pd.get_dummies(data_to_encode,drop_first = True).columns
        self.encoder.fit(data_to_encode)
        return self.encoder
    
    def transform(self,X_test):
        
        data = X_test.copy()
        data.reset_index(drop = True,inplace =True)
        data_to_encode = data[self.features_to_encode]
        data_left = data.drop(self.features_to_encode,axis = 1)
        
        data_encoded = pd.DataFrame(self.encoder.transform(data_to_encode),columns = self.columns)
        
        return pd.concat([data_left,data_encoded],axis = 1)

In [228]:

app = Flask(__name__)

def safe_paths():
    '''
    Function helper to define path of pickled model and preprocessed dataframes.
    '''
    
    model_path = os.path.join('model_pickle.pkl')
    train_path = os.path.join('df_encoded_for_train.pkl')
    ref_path = os.path.join('df_encoded_for_ref.pkl')
    
    return (model_path, train_path, ref_path)


def load_models():
    '''
    Helper function to load pickled model and preprocessed dataframes
    '''
    model_path, train_path, ref_path = safe_paths()
    
    pickled_model = pickle.load(open(model_path, 'rb'))
    pickled_train = pickle.load(open(train_path, 'rb'))
    pickled_ref = pickle.load(open(ref_path, 'rb'))
        
    return (pickled_model, pickled_train, pickled_ref)  


def getList(dict_input = {"track_id": {"0":"6jvvpPJQJy5rMOEkLlADl6"}}):
    '''
    Takes in dictionary such as shown below
    sample_input = {"track_id": {"0":"6jvvpPJQJy5rMOEkLlADl6"}}
    outputs a dictionaty object
    '''
    value = list(dict_input.values())[0]
    return value


def key_value(dict_input):
    '''
    Separates the key and value(track_id)
    '''
    value = getList(getList(dict_input))
    return value

############################################################
'Saving loaded pickled files in global memory'

PICKLED_MODEL, PICKLED_TRAIN, PICKLED_REF = load_models()
#############################################################


def recommend_track_id(track_id = key_value(dict_input)):
    '''
    Helper function that takes in atrack_id and recommends 14 track_ids of similar songs
    '''
    
    song_num = PICKLED_REF[PICKLED_REF['track_id']==track_id].song_num.to_numpy()[0]
    distance, neighbors = PICKLED_MODEL.kneighbors(np.array([PICKLED_TRAIN.values[song_num]]))
    
    song_list = []
    for item in neighbors[0][1:]: # this way excludes itself
        row = PICKLED_REF.iloc[item]
        song_list.append((row.track_id))  # print list of track id
      
    return song_list


def recommend_json(track_id='2YegxR5As7BeQuVp2U6pek'):
    '''
    Helper function the returns track_ids in json format.
    '''
    
    rec_list = recommend_track_id(track_id)
    rec_df = pd.DataFrame(rec_list, columns=['rec_track_id'])
    rec_json = rec_df.to_json()
    
    return rec_json
    

# @Pickled_model.route('/path1', methods=["POST"])      # from track_id user input
def parse_list(payload= {"track_id": {"0":"6jvvpPJQJy5rMOEkLlADl6"}}):
    '''
    Takes in json-like input, 
    {"track_id": {"0":"6jvvpPJQJy5rMOEkLlADl6"}}
    Outputs a list of track_id.
    '''
    # Fetching dictionary like user input
    #payload = request.get_json()    #Comment out until connected to API
        
    # Getting the dictionary
    trackid_dict = getList(payload)
    
    # Pulling out track ID
    trackid = key_value(payload)
    
    # Getting recommendations
    rec_list = recommend_track_id(trackid)
    
    return rec_list   


# @Pickled_model.route('/path2', methods=["POST"])      # from track_id user input
def parse_json(payload= {"track_id": {"0":"6jvvpPJQJy5rMOEkLlADl6"}}):
    '''
    Takes in json-like input, 
    {"track_id": {"0":"6jvvpPJQJy5rMOEkLlADl6"}}
    Outputs a json object just the input.
    '''
    # Fetching dictionary like user input
   # payload = request.get_json()   #Comment out until connected to API
        
    # Getting the dictionary
    trackid_dict = getList(payload)
    
    # Pulling out track ID
    trackid = key_value(payload)
    
    #Getting recommendations
    rec_json = recommend_track_id(trackid)
    
    return rec_json   
    

In [235]:
parse_list({"track_id":{'1':'3K4HG9evC7dg3N0R9cYqk4'}})

['0T1hQWoeHLABtNE0gpPCd0',
 '4wAB6We9cAzhj5biR2hXnh',
 '5O8eCRZGcjLfztSye87CMw',
 '2vLWF4yFhTHQZVwrPTjH1s',
 '4j4pPKE3xAblPIbhxScC1j',
 '2fwkl5Z8ujDtAh6qxEA5x0',
 '5mkGfmJGFZpwK9nA5amOhv',
 '1VGzxJnVQND7Cg5H5wGj14',
 '0rj3snMN7CFiYaLIVSegml',
 '4prEPl61C8qZpeo3IkYSMl',
 '5Tu2cCkMNQG8cDvTOOLzqE',
 '5Dx5nWIf6QKWjLsLUCXoml',
 '0KKist1jJYO17nOqtO1sb2',
 '1OXDtQ2nOR3Mvxx7k3xwNl']

In [237]:
parse_json()

['2vDfGmj1miRaaTD25N31bR',
 '5fqQTiZGv0mY6ZspmjY9Im',
 '2Asiem9O0YGuvvoAkz8aSq',
 '3K4HG9evC7dg3N0R9cYqk4',
 '3awuorsQsio2PUOvzq3hpd',
 '5BoDFGTXZ8j5muc6Sb2jMz',
 '45uDtD6s3y0V4tkWZrPDXy',
 '4wJ8dJKBAxdDe5cUx8K779',
 '4o7WXic8yAQWOV7iJXZ4jD',
 '17yrCsl1Ai6CZLBmGj6d6p',
 '2OMqSRS11ly56zg9u5za6c',
 '6dYCiAzt1o9D1XeEVrID1B',
 '6eLVgH7wafOJeHPzrzigrO',
 '2sq6zVcw0zr5go8ZqaUdUF']